In [ ]:
pip install pydicom nibabel numpy torch torchvision segmentation-models-pytorch scikit-learn

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install albumentations

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install pandas

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install opencv-python

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os
import numpy as np
import pydicom
import nibabel as nib
import torch
from torch.utils.data import DataLoader, Dataset
import albumentations as A
from albumentations.pytorch import ToTensorV2
import cv2
import pandas as pd
import random
import segmentation_models_pytorch as smp
import torch.optim as optim
import torch.nn as nn
from sklearn.metrics import jaccard_score

random.seed(42)
np.random.seed(42)
torch.manual_seed(42)


num_classes = 7
batch_size = 4
img_root = "/home/ealam/JHIR_Hip_Knee_Datasets/Hip/Images"
mask_root = "/home/ealam/JHIR_Hip_Knee_Datasets/Hip/Annotations"
metadata_path = "/home/ealam/JHIR_Hip_Knee_Datasets/Hip/segmentation_with_racegender.csv"


test_augmentations = A.Compose([
    A.Resize(height=512, width=512),
    A.Normalize(mean=(0.485, 0.485, 0.485), std=(0.229, 0.229, 0.229)),
    ToTensorV2(),
])

class MulticlassHipSegmentationDataset(Dataset):
    def __init__(self, img_root, mask_root, metadata_df, paired_files, num_classes, transforms=None, preprocessing=None):
        self.img_root = img_root
        self.mask_root = mask_root
        self.metadata_df = metadata_df
        self.paired_files = paired_files
        self.num_classes = num_classes
        self.transforms = transforms
        self.preprocessing = preprocessing

    def __len__(self):
        return len(self.paired_files)

    def __getitem__(self, idx):
        image_file, mask_file = self.paired_files[idx]
        if not os.path.exists(os.path.join(self.mask_root, mask_file)):
            return None

        dicom_image = pydicom.dcmread(os.path.join(self.img_root, image_file))
        image = dicom_image.pixel_array.astype(np.float32)
        image = cv2.resize(image, (512, 512))


        image = np.stack([image] * 3, axis=-1)

        annotation = nib.load(os.path.join(self.mask_root, mask_file))
        annotation_data = annotation.get_fdata()
        if len(annotation_data.shape) == 3:
            annotation_data = annotation_data[:, :, 0]

        annotation_data = self.calculate_flipped_rotated_mask(annotation_data)
        annotation_data = cv2.resize(annotation_data,

        if self.transforms is not None:
            transformed = self.transforms(image=image, mask=annotation_data)
            image = transformed["image"]
            annotation_data = transformed["mask"]

        annotation_data_onehot = self.one_hot_encode(annotation_data)

        if self.preprocessing is not None:
            transformed = self.preprocessing(image=image, mask=annotation_data_onehot)
            image = transformed["image"]
            annotation_data_onehot = transformed["mask"]

        patient_id = int(float(image_file.split(".")[0]))
        racegender_info = self.metadata_df.loc[self.metadata_df['id'] == patient_id]['racegender'].values
        racegender = racegender_info[0] if racegender_info.size > 0 else 'Unknown'

        return image, annotation_data_onehot, racegender

    def one_hot_encode(self, mask):
        one_hot_mask = np.zeros((self.num_classes, *mask.shape), dtype=np.float32)
        for class_idx in range(self.num_classes):
            one_hot_mask[class_idx][mask == class_idx] = 1.0
        return one_hot_mask

    def calculate_flipped_rotated_mask(self, mask):
        rotated_mask = cv2.rotate(mask, cv2.ROTATE_90_CLOCKWISE)
        flipped_rotated_mask = cv2.flip(rotated_mask, 1)
        return flipped_rotated_mask

metadata_df = pd.read_csv(metadata_path)

image_files = sorted(os.listdir(img_root))
mask_files = sorted(os.listdir(mask_root))

paired_files = []

for image_file in image_files:
    image_id = os.path.splitext(image_file)[0]
    mask_file = f"{image_id}.nii.gz"
    if mask_file in mask_files:
        paired_files.append((image_file, mask_file))

random.shuffle(paired_files)

train_size = int(0.7 * len(paired_files))
valid_size = int(0.1 * len(paired_files))
test_size = len(paired_files) - train_size - valid_size

train_pairs = paired_files[:train_size]
valid_pairs = paired_files[train_size:train_size + valid_size]
test_pairs = paired_files[train_size + valid_size:]

train_set = MulticlassHipSegmentationDataset(
    img_root, mask_root, metadata_df, train_pairs, num_classes,
    transforms=test_augmentations
)

def train_unetplusplus_model(num_epochs=100, encoder_name="resnet18"):

    model = smp.UnetPlusPlus(
        encoder_name=encoder_name,
        encoder_weights="imagenet",
        in_channels=3,
        classes=num_classes,
    )

    device = torch.device("cpu")
    model.to(device)

    optimizer = optim.Adam(model.parameters(), lr=0.0001, weight_decay=1e-5)
    criterion = nn.CrossEntropyLoss()

    valid_set = MulticlassHipSegmentationDataset(
        img_root, mask_root, metadata_df, valid_pairs, num_classes,
        transforms=test_augmentations
    )

    valid_loader = DataLoader(valid_set, batch_size=batch_size, shuffle=False, num_workers=2)

    for epoch in range(num_epochs):
        train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=2)

        model.train()
        train_iou_list = []

        for batch_idx, (images, masks, racegender) in enumerate(train_loader):
            if images is None:
                continue

            images, masks = images.to(device), masks.to(device)
            optimizer.zero_grad()
            outputs = model(images)

            predicted_masks = torch.argmax(outputs, dim=1)
            loss = criterion(outputs, masks.argmax(dim=1))
            loss.backward()
            optimizer.step()

            train_iou = jaccard_score(
                masks.argmax(dim=1).cpu().numpy().flatten(),
                predicted_masks.cpu().numpy().flatten(),
                average='micro'
            )
            train_iou_list.append(train_iou)

        model.eval()
        valid_iou_list = []

        for batch_idx, (images, masks, racegender) in enumerate(valid_loader):
            images, masks = images.to(device), masks.to(device)
            with torch.no_grad():
                outputs = model(images)
            predicted_masks = torch.argmax(outputs, dim=1)

            valid_iou = jaccard_score(
                masks.argmax(dim=1).cpu().numpy().flatten(),
                predicted_masks.cpu().numpy().flatten(),
                average='micro'
            )

            valid_iou_list.append(valid_iou)

        train_iou_avg = np.mean(train_iou_list)
        valid_iou_avg = np.mean(valid_iou_list)

        print(f"Epoch [{epoch + 1}/{num_epochs}] - Train IoU: {train_iou_avg:.4f} - Validation IoU: {valid_iou_avg:.4f}")


    torch.save(model.state_dict(), 'unetplusplus_model_hip_init.pth')


    test_set = MulticlassHipSegmentationDataset(
        img_root, mask_root, metadata_df, test_pairs, num_classes,
        transforms=test_augmentations
    )

    test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False, num_workers=2)

    model.eval()
    test_iou_list = []

    for batch_idx, (images, masks, racegender) in enumerate(test_loader):
        images, masks = images.to(device), masks.to(device)
        with torch.no_grad():
            outputs = model(images)
        predicted_masks = torch.argmax(outputs, dim=1)

        test_iou = jaccard_score(
            masks.argmax(dim=1).cpu().numpy().flatten(),
            predicted_masks.cpu().numpy().flatten(),
            average='micro'
        )

        test_iou_list.append(test_iou)

    test_iou_avg = np.mean(test_iou_list)

    print("Model saved successfully.")


print("Training U-Net++ ..")
train_unetplusplus_model(encoder_name="resnet18")


Training U-Net++ ..
Epoch [1/100] - Train IoU: 0.0598 - Validation IoU: 0.0230
Epoch [2/100] - Train IoU: 0.1014 - Validation IoU: 0.0476
Epoch [3/100] - Train IoU: 0.1478 - Validation IoU: 0.1551
Epoch [4/100] - Train IoU: 0.1827 - Validation IoU: 0.1813
Epoch [5/100] - Train IoU: 0.2550 - Validation IoU: 0.2145
Epoch [6/100] - Train IoU: 0.2893 - Validation IoU: 0.2580
Epoch [7/100] - Train IoU: 0.3284 - Validation IoU: 0.2994
Epoch [8/100] - Train IoU: 0.3981 - Validation IoU: 0.3358
Epoch [9/100] - Train IoU: 0.4370 - Validation IoU: 0.3420
Epoch [10/100] - Train IoU: 0.4905 - Validation IoU: 0.3795
Epoch [11/100] - Train IoU: 0.5195 - Validation IoU: 0.4407
Epoch [12/100] - Train IoU: 0.5749 - Validation IoU: 0.5048
Epoch [13/100] - Train IoU: 0.5954 - Validation IoU: 0.5585
Epoch [14/100] - Train IoU: 0.6409 - Validation IoU: 0.6014
Epoch [15/100] - Train IoU: 0.6727 - Validation IoU: 0.6511
Epoch [16/100] - Train IoU: 0.6929 - Validation IoU: 0.6862
Epoch [17/100] - Train IoU: 0

In [ ]:
import os
import numpy as np
import pydicom
import nibabel as nib
import torch
from torch.utils.data import DataLoader, Dataset
import albumentations as A
from albumentations.pytorch import ToTensorV2
import cv2
import pandas as pd
import random
import segmentation_models_pytorch as smp
import torch.optim as optim
import torch.nn as nn
from sklearn.metrics import jaccard_score


random.seed(42)
np.random.seed(42)
torch.manual_seed(42)


num_classes = 7
batch_size = 4
img_root = "/home/ealam/JHIR_Hip_Knee_Datasets/Hip/Images"
mask_root = "/home/ealam/JHIR_Hip_Knee_Datasets/Hip/Annotations"
metadata_path = "/home/ealam/JHIR_Hip_Knee_Datasets/Hip/segmentation_with_racegender.csv"


test_augmentations = A.Compose([
    A.Resize(height=512, width=512),
    A.Normalize(mean=(0.485, 0.485, 0.485), std=(0.229, 0.229, 0.229)),
    ToTensorV2(),
])

class MulticlassHipSegmentationDataset(Dataset):
    def __init__(self, img_root, mask_root, metadata_df, paired_files, num_classes, transforms=None, preprocessing=None):
        self.img_root = img_root
        self.mask_root = mask_root
        self.metadata_df = metadata_df
        self.paired_files = paired_files
        self.num_classes = num_classes
        self.transforms = transforms
        self.preprocessing = preprocessing

    def __len__(self):
        return len(self.paired_files)

    def __getitem__(self, idx):
        image_file, mask_file = self.paired_files[idx]
        if not os.path.exists(os.path.join(self.mask_root, mask_file)):
            return None

        dicom_image = pydicom.dcmread(os.path.join(self.img_root, image_file))
        image = dicom_image.pixel_array.astype(np.float32)
        image = cv2.resize(image, (512, 512))


        image = np.stack([image] * 3, axis=-1)

        annotation = nib.load(os.path.join(self.mask_root, mask_file))
        annotation_data = annotation.get_fdata()
        if len(annotation_data.shape) == 3:
            annotation_data = annotation_data[:, :, 0]

        annotation_data = self.calculate_flipped_rotated_mask(annotation_data)
        annotation_data = cv2.resize(annotation_data, (512, 512))

        if self.transforms is not None:
            transformed = self.transforms(image=image, mask=annotation_data)
            image = transformed["image"]
            annotation_data = transformed["mask"]

        annotation_data_onehot = self.one_hot_encode(annotation_data)

        if self.preprocessing is not None:
            transformed = self.preprocessing(image=image, mask=annotation_data_onehot)
            image = transformed["image"]
            annotation_data_onehot = transformed["mask"]

        patient_id = int(float(image_file.split(".")[0]))
        racegender_info = self.metadata_df.loc[self.metadata_df['id'] == patient_id]['racegender'].values
        racegender = racegender_info[0] if racegender_info.size > 0 else 'Unknown'

        return image, annotation_data_onehot, racegender

    def one_hot_encode(self, mask):
        one_hot_mask = np.zeros((self.num_classes, *mask.shape), dtype=np.float32)
        for class_idx in range(self.num_classes):
            one_hot_mask[class_idx][mask == class_idx] = 1.0
        return one_hot_mask

    def calculate_flipped_rotated_mask(self, mask):
        rotated_mask = cv2.rotate(mask, cv2.ROTATE_90_CLOCKWISE)
        flipped_rotated_mask = cv2.flip(rotated_mask, 1)
        return flipped_rotated_mask

metadata_df = pd.read_csv(metadata_path)

image_files = sorted(os.listdir(img_root))
mask_files = sorted(os.listdir(mask_root))

paired_files = []

for image_file in image_files:
    image_id = os.path.splitext(image_file)[0]
    mask_file = f"{image_id}.nii.gz"
    if mask_file in mask_files:
        paired_files.append((image_file, mask_file))

random.shuffle(paired_files)

train_size = int(0.7 * len(paired_files))
valid_size = int(0.1 * len(paired_files))
test_size = len(paired_files) - train_size - valid_size

train_pairs = paired_files[:train_size]
valid_pairs = paired_files[train_size:train_size + valid_size]
test_pairs = paired_files[train_size + valid_size:]

train_set = MulticlassHipSegmentationDataset(
    img_root, mask_root, metadata_df, train_pairs, num_classes,
    transforms=test_augmentations
)

def train_linknet_model(num_epochs=100, encoder_name="resnet18"):

    model = smp.Linknet(
        encoder_name=encoder_name,
        encoder_weights="imagenet",
        in_channels=3,
        classes=num_classes,
    )

    device = torch.device("cpu")
    model.to(device)

    optimizer = optim.Adam(model.parameters(), lr=0.0001, weight_decay=1e-5)
    criterion = nn.CrossEntropyLoss()

    valid_set = MulticlassHipSegmentationDataset(
        img_root, mask_root, metadata_df, valid_pairs, num_classes,
        transforms=test_augmentations
    )

    valid_loader = DataLoader(valid_set, batch_size=batch_size, shuffle=False, num_workers=2)

    for epoch in range(num_epochs):
        train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=2)

        model.train()
        train_iou_list = []

        for batch_idx, (images, masks, racegender) in enumerate(train_loader):
            if images is None:
                continue

            images, masks = images.to(device), masks.to(device)
            optimizer.zero_grad()
            outputs = model(images)

            predicted_masks = torch.argmax(outputs, dim=1)
            loss = criterion(outputs, masks.argmax(dim=1))
            loss.backward()
            optimizer.step()

            train_iou = jaccard_score(
                masks.argmax(dim=1).cpu().numpy().flatten(),
                predicted_masks.cpu().numpy().flatten(),
                average='micro'
            )
            train_iou_list.append(train_iou)

        model.eval()
        valid_iou_list = []

        for batch_idx, (images, masks, racegender) in enumerate(valid_loader):
            images, masks = images.to(device), masks.to(device)
            with torch.no_grad():
                outputs = model(images)
            predicted_masks = torch.argmax(outputs, dim=1)

            valid_iou = jaccard_score(
                masks.argmax(dim=1).cpu().numpy().flatten(),
                predicted_masks.cpu().numpy().flatten(),
                average='micro'
            )

            valid_iou_list.append(valid_iou)

        train_iou_avg = np.mean(train_iou_list)
        valid_iou_avg = np.mean(valid_iou_list)

        print(f"Epoch [{epoch + 1}/{num_epochs}] - Train IoU: {train_iou_avg:.4f} - Validation IoU: {valid_iou_avg:.4f}")


    torch.save(model.state_dict(), 'linknet_model_Hip_init.pth')


    test_set = MulticlassHipSegmentationDataset(
        img_root, mask_root, metadata_df, test_pairs, num_classes,
        transforms=test_augmentations
    )

    test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False, num_workers=2)

    model.eval()
    test_iou_list = []

    for batch_idx, (images, masks, racegender) in enumerate(test_loader):
        images, masks = images.to(device), masks.to(device)
        with torch.no_grad():
            outputs = model(images)
        predicted_masks = torch.argmax(outputs, dim=1)

        test_iou = jaccard_score(
            masks.argmax(dim=1).cpu().numpy().flatten(),
            predicted_masks.cpu().numpy().flatten(),
            average='micro'
        )

        test_iou_list.append(test_iou)

    test_iou_avg = np.mean(test_iou_list)


    print("Model saved successfully.")


print("Training LinkNet ..")
train_linknet_model(encoder_name="resnet18")


Training LinkNet ..
Epoch [1/100] - Train IoU: 0.0806 - Validation IoU: 0.0288
Epoch [2/100] - Train IoU: 0.0883 - Validation IoU: 0.0280
Epoch [3/100] - Train IoU: 0.1000 - Validation IoU: 0.0278
Epoch [4/100] - Train IoU: 0.1096 - Validation IoU: 0.0271
Epoch [5/100] - Train IoU: 0.1252 - Validation IoU: 0.0259
Epoch [6/100] - Train IoU: 0.1236 - Validation IoU: 0.0254
Epoch [7/100] - Train IoU: 0.1465 - Validation IoU: 0.0327
Epoch [8/100] - Train IoU: 0.1529 - Validation IoU: 0.0499
Epoch [9/100] - Train IoU: 0.1578 - Validation IoU: 0.0848
Epoch [10/100] - Train IoU: 0.1657 - Validation IoU: 0.1133
Epoch [11/100] - Train IoU: 0.1711 - Validation IoU: 0.1286
Epoch [12/100] - Train IoU: 0.1696 - Validation IoU: 0.1349
Epoch [13/100] - Train IoU: 0.1826 - Validation IoU: 0.1446
Epoch [14/100] - Train IoU: 0.1891 - Validation IoU: 0.1507
Epoch [15/100] - Train IoU: 0.1878 - Validation IoU: 0.1548
Epoch [16/100] - Train IoU: 0.1843 - Validation IoU: 0.1546
Epoch [17/100] - Train IoU: 0

In [ ]:
import os
import numpy as np
import pydicom
import nibabel as nib
import torch
from torch.utils.data import DataLoader, Dataset
import albumentations as A
from albumentations.pytorch import ToTensorV2
import cv2
import pandas as pd
import random
import segmentation_models_pytorch as smp
import torch.optim as optim
import torch.nn as nn
from sklearn.metrics import jaccard_score


random.seed(42)
np.random.seed(42)
torch.manual_seed(42)


num_classes = 7
batch_size = 4
img_root = "/home/ealam/JHIR_Hip_Knee_Datasets/Hip/Images"
mask_root = "/home/ealam/JHIR_Hip_Knee_Datasets/Hip/Annotations"
metadata_path = "/home/ealam/JHIR_Hip_Knee_Datasets/Hip/segmentation_with_racegender.csv"


test_augmentations = A.Compose([
    A.Resize(height=512, width=512),
    A.Normalize(mean=(0.485, 0.485, 0.485), std=(0.229, 0.229, 0.229)),
    ToTensorV2(),
])

class MulticlassHipSegmentationDataset(Dataset):
    def __init__(self, img_root, mask_root, metadata_df, paired_files, num_classes, transforms=None, preprocessing=None):
        self.img_root = img_root
        self.mask_root = mask_root
        self.metadata_df = metadata_df
        self.paired_files = paired_files
        self.num_classes = num_classes
        self.transforms = transforms
        self.preprocessing = preprocessing

    def __len__(self):
        return len(self.paired_files)

    def __getitem__(self, idx):
        image_file, mask_file = self.paired_files[idx]
        if not os.path.exists(os.path.join(self.mask_root, mask_file)):
            return None

        dicom_image = pydicom.dcmread(os.path.join(self.img_root, image_file))
        image = dicom_image.pixel_array.astype(np.float32)
        image = cv2.resize(image, (512, 512))


        image = np.stack([image] * 3, axis=-1)

        annotation = nib.load(os.path.join(self.mask_root, mask_file))
        annotation_data = annotation.get_fdata()
        if len(annotation_data.shape) == 3:
            annotation_data = annotation_data[:, :, 0]

        annotation_data = self.calculate_flipped_rotated_mask(annotation_data)
        annotation_data = cv2.resize(annotation_data, (512, 512))

        if self.transforms is not None:
            transformed = self.transforms(image=image, mask=annotation_data)
            image = transformed["image"]
            annotation_data = transformed["mask"]

        annotation_data_onehot = self.one_hot_encode(annotation_data)

        if self.preprocessing is not None:
            transformed = self.preprocessing(image=image, mask=annotation_data_onehot)
            image = transformed["image"]
            annotation_data_onehot = transformed["mask"]

        patient_id = int(float(image_file.split(".")[0]))
        racegender_info = self.metadata_df.loc[self.metadata_df['id'] == patient_id]['racegender'].values
        racegender = racegender_info[0] if racegender_info.size > 0 else 'Unknown'

        return image, annotation_data_onehot, racegender

    def one_hot_encode(self, mask):
        one_hot_mask = np.zeros((self.num_classes, *mask.shape), dtype=np.float32)
        for class_idx in range(self.num_classes):
            one_hot_mask[class_idx][mask == class_idx] = 1.0
        return one_hot_mask

    def calculate_flipped_rotated_mask(self, mask):
        rotated_mask = cv2.rotate(mask, cv2.ROTATE_90_CLOCKWISE)
        flipped_rotated_mask = cv2.flip(rotated_mask, 1)
        return flipped_rotated_mask

metadata_df = pd.read_csv(metadata_path)

image_files = sorted(os.listdir(img_root))
mask_files = sorted(os.listdir(mask_root))

paired_files = []

for image_file in image_files:
    image_id = os.path.splitext(image_file)[0]
    mask_file = f"{image_id}.nii.gz"
    if mask_file in mask_files:
        paired_files.append((image_file, mask_file))

random.shuffle(paired_files)

train_size = int(0.7 * len(paired_files))
valid_size = int(0.1 * len(paired_files))
test_size = len(paired_files) - train_size - valid_size

train_pairs = paired_files[:train_size]
valid_pairs = paired_files[train_size:train_size + valid_size]
test_pairs = paired_files[train_size + valid_size:]

train_set = MulticlassHipSegmentationDataset(
    img_root, mask_root, metadata_df, train_pairs, num_classes,
    transforms=test_augmentations
)

def train_pspnet_model(num_epochs=100, encoder_name="resnet18"):

    model = smp.PSPNet(
        encoder_name=encoder_name,
        encoder_weights="imagenet",
        in_channels=3,
        classes=num_classes,
    )

    device = torch.device("cpu")
    model.to(device)

    optimizer = optim.Adam(model.parameters(), lr=0.0001, weight_decay=1e-5)
    criterion = nn.CrossEntropyLoss()

    valid_set = MulticlassHipSegmentationDataset(
        img_root, mask_root, metadata_df, valid_pairs, num_classes,
        transforms=test_augmentations
    )

    valid_loader = DataLoader(valid_set, batch_size=batch_size, shuffle=False, num_workers=2)

    for epoch in range(num_epochs):
        train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=2)

        model.train()
        train_iou_list = []

        for batch_idx, (images, masks, racegender) in enumerate(train_loader):
            if images is None:
                continue

            images, masks = images.to(device), masks.to(device)
            optimizer.zero_grad()
            outputs = model(images)

            predicted_masks = torch.argmax(outputs, dim=1)
            loss = criterion(outputs, masks.argmax(dim=1))
            loss.backward()
            optimizer.step()

            train_iou = jaccard_score(
                masks.argmax(dim=1).cpu().numpy().flatten(),
                predicted_masks.cpu().numpy().flatten(),
                average='micro'
            )
            train_iou_list.append(train_iou)

        model.eval()
        valid_iou_list = []

        for batch_idx, (images, masks, racegender) in enumerate(valid_loader):
            images, masks = images.to(device), masks.to(device)
            with torch.no_grad():
                outputs = model(images)
            predicted_masks = torch.argmax(outputs, dim=1)

            valid_iou = jaccard_score(
                masks.argmax(dim=1).cpu().numpy().flatten(),
                predicted_masks.cpu().numpy().flatten(),
                average='micro'
            )

            valid_iou_list.append(valid_iou)

        train_iou_avg = np.mean(train_iou_list)
        valid_iou_avg = np.mean(valid_iou_list)

        print(f"Epoch [{epoch + 1}/{num_epochs}] - Train IoU: {train_iou_avg:.4f} - Validation IoU: {valid_iou_avg:.4f}")


    torch.save(model.state_dict(), 'pspnet_model_Hip_init.pth')


    test_set = MulticlassHipSegmentationDataset(
        img_root, mask_root, metadata_df, test_pairs, num_classes,
        transforms=test_augmentations
    )

    test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False, num_workers=2)

    model.eval()
    test_iou_list = []

    for batch_idx, (images, masks, racegender) in enumerate(test_loader):
        images, masks = images.to(device), masks.to(device)
        with torch.no_grad():
            outputs = model(images)
        predicted_masks = torch.argmax(outputs, dim=1)

        test_iou = jaccard_score(
            masks.argmax(dim=1).cpu().numpy().flatten(),
            predicted_masks.cpu().numpy().flatten(),
            average='micro'
        )

        test_iou_list.append(test_iou)

    test_iou_avg = np.mean(test_iou_list)

    print("Model saved successfully.")


print("Training PSPNet ..")
train_pspnet_model(encoder_name="resnet18")

Training PSPNet ..
Epoch [1/100] - Train IoU: 0.1508 - Validation IoU: 0.4623
Epoch [2/100] - Train IoU: 0.3613 - Validation IoU: 0.5369
Epoch [3/100] - Train IoU: 0.4850 - Validation IoU: 0.5392
Epoch [4/100] - Train IoU: 0.5449 - Validation IoU: 0.5393
Epoch [5/100] - Train IoU: 0.5661 - Validation IoU: 0.5393
Epoch [6/100] - Train IoU: 0.5691 - Validation IoU: 0.5393
Epoch [7/100] - Train IoU: 0.5418 - Validation IoU: 0.5393
Epoch [8/100] - Train IoU: 0.5772 - Validation IoU: 0.5393
Epoch [9/100] - Train IoU: 0.5477 - Validation IoU: 0.5393
Epoch [10/100] - Train IoU: 0.5676 - Validation IoU: 0.5397
Epoch [11/100] - Train IoU: 0.5873 - Validation IoU: 0.5408
Epoch [12/100] - Train IoU: 0.6059 - Validation IoU: 0.5448
Epoch [13/100] - Train IoU: 0.6239 - Validation IoU: 0.5568
Epoch [14/100] - Train IoU: 0.6347 - Validation IoU: 0.5769
Epoch [15/100] - Train IoU: 0.6617 - Validation IoU: 0.6027
Epoch [16/100] - Train IoU: 0.6584 - Validation IoU: 0.6296
Epoch [17/100] - Train IoU: 0.

In [ ]:
import os
import numpy as np
import pydicom
import nibabel as nib
import torch
from torch.utils.data import DataLoader, Dataset
import albumentations as A
from albumentations.pytorch import ToTensorV2
import cv2
import pandas as pd
import random
import segmentation_models_pytorch as smp
import torch.optim as optim
import torch.nn as nn
from sklearn.metrics import jaccard_score


random.seed(42)
np.random.seed(42)
torch.manual_seed(42)

num_classes = 7
batch_size = 4
img_root = "/home/ealam/JHIR_Hip_Knee_Datasets/Hip/Images"
mask_root = "/home/ealam/JHIR_Hip_Knee_Datasets/Hip/Annotations"
metadata_path = "/home/ealam/JHIR_Hip_Knee_Datasets/Hip/segmentation_with_racegender.csv"


test_augmentations = A.Compose([
    A.Resize(height=512, width=512),
    A.Normalize(mean=(0.485, 0.485, 0.485), std=(0.229, 0.229, 0.229)),
    ToTensorV2(),
])

class MulticlassHipSegmentationDataset(Dataset):
    def __init__(self, img_root, mask_root, metadata_df, paired_files, num_classes, transforms=None, preprocessing=None):
        self.img_root = img_root
        self.mask_root = mask_root
        self.metadata_df = metadata_df
        self.paired_files = paired_files
        self.num_classes = num_classes
        self.transforms = transforms
        self.preprocessing = preprocessing

    def __len__(self):
        return len(self.paired_files)

    def __getitem__(self, idx):
        image_file, mask_file = self.paired_files[idx]
        if not os.path.exists(os.path.join(self.mask_root, mask_file)):
            return None

        dicom_image = pydicom.dcmread(os.path.join(self.img_root, image_file))
        image = dicom_image.pixel_array.astype(np.float32)
        image = cv2.resize(image, (512, 512))


        image = np.stack([image] * 3, axis=-1)

        annotation = nib.load(os.path.join(self.mask_root, mask_file))
        annotation_data = annotation.get_fdata()
        if len(annotation_data.shape) == 3:
            annotation_data = annotation_data[:, :, 0]

        annotation_data = self.calculate_flipped_rotated_mask(annotation_data)
        annotation_data = cv2.resize(annotation_data, (512, 512))

        if self.transforms is not None:
            transformed = self.transforms(image=image, mask=annotation_data)
            image = transformed["image"]
            annotation_data = transformed["mask"]

        annotation_data_onehot = self.one_hot_encode(annotation_data)

        if self.preprocessing is not None:
            transformed = self.preprocessing(image=image, mask=annotation_data_onehot)
            image = transformed["image"]
            annotation_data_onehot = transformed["mask"]

        patient_id = int(float(image_file.split(".")[0]))
        racegender_info = self.metadata_df.loc[self.metadata_df['id'] == patient_id]['racegender'].values
        racegender = racegender_info[0] if racegender_info.size > 0 else 'Unknown'

        return image, annotation_data_onehot, racegender

    def one_hot_encode(self, mask):
        one_hot_mask = np.zeros((self.num_classes, *mask.shape), dtype=np.float32)
        for class_idx in range(self.num_classes):
            one_hot_mask[class_idx][mask == class_idx] = 1.0
        return one_hot_mask

    def calculate_flipped_rotated_mask(self, mask):
        rotated_mask = cv2.rotate(mask, cv2.ROTATE_90_CLOCKWISE)
        flipped_rotated_mask = cv2.flip(rotated_mask, 1)
        return flipped_rotated_mask

metadata_df = pd.read_csv(metadata_path)

image_files = sorted(os.listdir(img_root))
mask_files = sorted(os.listdir(mask_root))

paired_files = []

for image_file in image_files:
    image_id = os.path.splitext(image_file)[0]
    mask_file = f"{image_id}.nii.gz"
    if mask_file in mask_files:
        paired_files.append((image_file, mask_file))

random.shuffle(paired_files)

train_size = int(0.7 * len(paired_files))
valid_size = int(0.1 * len(paired_files))
test_size = len(paired_files) - train_size - valid_size

train_pairs = paired_files[:train_size]
valid_pairs = paired_files[train_size:train_size + valid_size]
test_pairs = paired_files[train_size + valid_size:]

train_set = MulticlassHipSegmentationDataset(
    img_root, mask_root, metadata_df, train_pairs, num_classes,
    transforms=test_augmentations
)

def train_fpn_model(num_epochs=100, encoder_name="resnet18"):

    model = smp.FPN(
        encoder_name=encoder_name,
        encoder_weights="imagenet",
        in_channels=3,
        classes=num_classes,
    )

    device = torch.device("cpu")
    model.to(device)

    optimizer = optim.Adam(model.parameters(), lr=0.0001, weight_decay=1e-5)
    criterion = nn.CrossEntropyLoss()

    valid_set = MulticlassHipSegmentationDataset(
        img_root, mask_root, metadata_df, valid_pairs, num_classes,
        transforms=test_augmentations
    )

    valid_loader = DataLoader(valid_set, batch_size=batch_size, shuffle=False, num_workers=2)

    for epoch in range(num_epochs):
        train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=2)

        model.train()
        train_iou_list = []

        for batch_idx, (images, masks, racegender) in enumerate(train_loader):
            if images is None:
                continue

            images, masks = images.to(device), masks.to(device)
            optimizer.zero_grad()
            outputs = model(images)

            predicted_masks = torch.argmax(outputs, dim=1)
            loss = criterion(outputs, masks.argmax(dim=1))
            loss.backward()
            optimizer.step()

            train_iou = jaccard_score(
                masks.argmax(dim=1).cpu().numpy().flatten(),
                predicted_masks.cpu().numpy().flatten(),
                average='micro'
            )
            train_iou_list.append(train_iou)

        model.eval()
        valid_iou_list = []

        for batch_idx, (images, masks, racegender) in enumerate(valid_loader):
            images, masks = images.to(device), masks.to(device)
            with torch.no_grad():
                outputs = model(images)
            predicted_masks = torch.argmax(outputs, dim=1)

            valid_iou = jaccard_score(
                masks.argmax(dim=1).cpu().numpy().flatten(),
                predicted_masks.cpu().numpy().flatten(),
                average='micro'
            )

            valid_iou_list.append(valid_iou)

        train_iou_avg = np.mean(train_iou_list)
        valid_iou_avg = np.mean(valid_iou_list)

        print(f"Epoch [{epoch + 1}/{num_epochs}] - Train IoU: {train_iou_avg:.4f} - Validation IoU: {valid_iou_avg:.4f}")


    torch.save(model.state_dict(), 'fpn_model_Hip_init.pth')


    test_set = MulticlassHipSegmentationDataset(
        img_root, mask_root, metadata_df, test_pairs, num_classes,
        transforms=test_augmentations
    )

    test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False, num_workers=2)

    model.eval()
    test_iou_list = []

    for batch_idx, (images, masks, racegender) in enumerate(test_loader):
        images, masks = images.to(device), masks.to(device)
        with torch.no_grad():
            outputs = model(images)
        predicted_masks = torch.argmax(outputs, dim=1)

        test_iou = jaccard_score(
            masks.argmax(dim=1).cpu().numpy().flatten(),
            predicted_masks.cpu().numpy().flatten(),
            average='micro'
        )

        test_iou_list.append(test_iou)

    test_iou_avg = np.mean(test_iou_list)

    print("Model saved successfully.")


print("Training FPN ..")
train_fpn_model(encoder_name="resnet18")


Training FPN ..
Epoch [1/100] - Train IoU: 0.1522 - Validation IoU: 0.5233
Epoch [2/100] - Train IoU: 0.5325 - Validation IoU: 0.5413
Epoch [3/100] - Train IoU: 0.6204 - Validation IoU: 0.5913
Epoch [4/100] - Train IoU: 0.6450 - Validation IoU: 0.6005
Epoch [5/100] - Train IoU: 0.6962 - Validation IoU: 0.6254
Epoch [6/100] - Train IoU: 0.7097 - Validation IoU: 0.6425
Epoch [7/100] - Train IoU: 0.7535 - Validation IoU: 0.6429
Epoch [8/100] - Train IoU: 0.7635 - Validation IoU: 0.6357
Epoch [9/100] - Train IoU: 0.7510 - Validation IoU: 0.6301
Epoch [10/100] - Train IoU: 0.7648 - Validation IoU: 0.6507
Epoch [11/100] - Train IoU: 0.7853 - Validation IoU: 0.6995
Epoch [12/100] - Train IoU: 0.8051 - Validation IoU: 0.7401
Epoch [13/100] - Train IoU: 0.7849 - Validation IoU: 0.7454
Epoch [14/100] - Train IoU: 0.8163 - Validation IoU: 0.7267
Epoch [15/100] - Train IoU: 0.8202 - Validation IoU: 0.7025
Epoch [16/100] - Train IoU: 0.8314 - Validation IoU: 0.6999
Epoch [17/100] - Train IoU: 0.815

In [ ]:
import os
import numpy as np
import pydicom
import nibabel as nib
import torch
from torch.utils.data import DataLoader, Dataset
import albumentations as A
from albumentations.pytorch import ToTensorV2
import cv2
import pandas as pd
import random
import segmentation_models_pytorch as smp
import torch.optim as optim
import torch.nn as nn
from sklearn.metrics import jaccard_score


random.seed(42)
np.random.seed(42)
torch.manual_seed(42)


num_classes = 7
batch_size = 4
img_root = "/home/ealam/JHIR_Hip_Knee_Datasets/Hip/Images"
mask_root = "/home/ealam/JHIR_Hip_Knee_Datasets/Hip/Annotations"
metadata_path = "/home/ealam/JHIR_Hip_Knee_Datasets/Hip/segmentation_with_racegender.csv"


test_augmentations = A.Compose([
    A.Resize(height=512, width=512),
    A.Normalize(mean=(0.485, 0.485, 0.485), std=(0.229, 0.229, 0.229)),
    ToTensorV2(),
])

class MulticlassHipSegmentationDataset(Dataset):
    def __init__(self, img_root, mask_root, metadata_df, paired_files, num_classes, transforms=None, preprocessing=None):
        self.img_root = img_root
        self.mask_root = mask_root
        self.metadata_df = metadata_df
        self.paired_files = paired_files
        self.num_classes = num_classes
        self.transforms = transforms
        self.preprocessing = preprocessing

    def __len__(self):
        return len(self.paired_files)

    def __getitem__(self, idx):
        image_file, mask_file = self.paired_files[idx]
        if not os.path.exists(os.path.join(self.mask_root, mask_file)):
            return None

        dicom_image = pydicom.dcmread(os.path.join(self.img_root, image_file))
        image = dicom_image.pixel_array.astype(np.float32)
        image = cv2.resize(image, (512, 512))


        image = np.stack([image] * 3, axis=-1)

        annotation = nib.load(os.path.join(self.mask_root, mask_file))
        annotation_data = annotation.get_fdata()
        if len(annotation_data.shape) == 3:
            annotation_data = annotation_data[:, :, 0]

        annotation_data = self.calculate_flipped_rotated_mask(annotation_data)
        annotation_data = cv2.resize(annotation_data, (512, 512))

        if self.transforms is not None:
            transformed = self.transforms(image=image, mask=annotation_data)
            image = transformed["image"]
            annotation_data = transformed["mask"]

        annotation_data_onehot = self.one_hot_encode(annotation_data)

        if self.preprocessing is not None:
            transformed = self.preprocessing(image=image, mask=annotation_data_onehot)
            image = transformed["image"]
            annotation_data_onehot = transformed["mask"]

        patient_id = int(float(image_file.split(".")[0]))
        racegender_info = self.metadata_df.loc[self.metadata_df['id'] == patient_id]['racegender'].values
        racegender = racegender_info[0] if racegender_info.size > 0 else 'Unknown'

        return image, annotation_data_onehot, racegender

    def one_hot_encode(self, mask):
        one_hot_mask = np.zeros((self.num_classes, *mask.shape), dtype=np.float32)
        for class_idx in range(self.num_classes):
            one_hot_mask[class_idx][mask == class_idx] = 1.0
        return one_hot_mask

    def calculate_flipped_rotated_mask(self, mask):
        rotated_mask = cv2.rotate(mask, cv2.ROTATE_90_CLOCKWISE)
        flipped_rotated_mask = cv2.flip(rotated_mask, 1)
        return flipped_rotated_mask

metadata_df = pd.read_csv(metadata_path)

image_files = sorted(os.listdir(img_root))
mask_files = sorted(os.listdir(mask_root))

paired_files = []

for image_file in image_files:
    image_id = os.path.splitext(image_file)[0]
    mask_file = f"{image_id}.nii.gz"
    if mask_file in mask_files:
        paired_files.append((image_file, mask_file))

random.shuffle(paired_files)

train_size = int(0.7 * len(paired_files))
valid_size = int(0.1 * len(paired_files))
test_size = len(paired_files) - train_size - valid_size

train_pairs = paired_files[:train_size]
valid_pairs = paired_files[train_size:train_size + valid_size]
test_pairs = paired_files[train_size + valid_size:]

train_set = MulticlassHipSegmentationDataset(
    img_root, mask_root, metadata_df, train_pairs, num_classes,
    transforms=test_augmentations
)

def train_pan_model(num_epochs=100, encoder_name="resnet18"):

    model = smp.PAN(
        encoder_name=encoder_name,
        encoder_weights="imagenet",
        in_channels=3,
        classes=num_classes,
    )

    device = torch.device("cpu")
    model.to(device)

    optimizer = optim.Adam(model.parameters(), lr=0.0001, weight_decay=1e-5)
    criterion = nn.CrossEntropyLoss()

    valid_set = MulticlassHipSegmentationDataset(
        img_root, mask_root, metadata_df, valid_pairs, num_classes,
        transforms=test_augmentations
    )

    valid_loader = DataLoader(valid_set, batch_size=batch_size, shuffle=False, num_workers=2)

    for epoch in range(num_epochs):
        train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=2)

        if len(train_loader) == 0:
            continue

        model.train()
        train_iou_list = []

        for batch_idx, (images, masks, racegender) in enumerate(train_loader):
            if images is None or masks is None:
                continue

            images, masks = images.to(device), masks.to(device)

            if images.size(0) < 2:
                continue

            optimizer.zero_grad()

            outputs = model(images)

            predicted_masks = torch.argmax(outputs, dim=1)
            loss = criterion(outputs, masks.argmax(dim=1))
            loss.backward()
            optimizer.step()

            train_iou = jaccard_score(
                masks.argmax(dim=1).cpu().numpy().flatten(),
                predicted_masks.cpu().numpy().flatten(),
                average='micro'
            )
            train_iou_list.append(train_iou)

        model.eval()
        valid_iou_list = []

        for batch_idx, (images, masks, racegender) in enumerate(valid_loader):
            if images is None or masks is None:
                continue

            images, masks = images.to(device), masks.to(device)
            with torch.no_grad():
                outputs = model(images)

            predicted_masks = torch.argmax(outputs, dim=1)

            valid_iou = jaccard_score(
                masks.argmax(dim=1).cpu().numpy().flatten(),
                predicted_masks.cpu().numpy().flatten(),
                average='micro'
            )

            valid_iou_list.append(valid_iou)

        train_iou_avg = np.mean(train_iou_list)
        valid_iou_avg = np.mean(valid_iou_list)

        print(f"Epoch {epoch + 1}/{num_epochs} - Training IoU: {train_iou_avg:.4f} - Validation IoU: {valid_iou_avg:.4f}")



    torch.save(model.state_dict(), "panhip_init.pth")
    print("Model saved ")


print("Training PAN ..")
train_pan_model(encoder_name="resnet18")


Training PAN ..
Epoch 1/100 - Training IoU: 0.4583 - Validation IoU: 0.5184
Epoch 2/100 - Training IoU: 0.4774 - Validation IoU: 0.5087
Epoch 3/100 - Training IoU: 0.5381 - Validation IoU: 0.4961
Epoch 4/100 - Training IoU: 0.5393 - Validation IoU: 0.4822
Epoch 5/100 - Training IoU: 0.5395 - Validation IoU: 0.5013
Epoch 6/100 - Training IoU: 0.5902 - Validation IoU: 0.5329
Epoch 7/100 - Training IoU: 0.6030 - Validation IoU: 0.5667
Epoch 8/100 - Training IoU: 0.6316 - Validation IoU: 0.5805
Epoch 9/100 - Training IoU: 0.6552 - Validation IoU: 0.5810
Epoch 10/100 - Training IoU: 0.6809 - Validation IoU: 0.5912
Epoch 11/100 - Training IoU: 0.7089 - Validation IoU: 0.6190
Epoch 12/100 - Training IoU: 0.7000 - Validation IoU: 0.6575
Epoch 13/100 - Training IoU: 0.7420 - Validation IoU: 0.6974
Epoch 14/100 - Training IoU: 0.7261 - Validation IoU: 0.7258
Epoch 15/100 - Training IoU: 0.7649 - Validation IoU: 0.7325
Epoch 16/100 - Training IoU: 0.7907 - Validation IoU: 0.7268
Epoch 17/100 - Tr

In [ ]:
import os
import numpy as np
import pydicom
import nibabel as nib
import torch
from torch.utils.data import DataLoader, Dataset
import albumentations as A
from albumentations.pytorch import ToTensorV2
import cv2
import pandas as pd
import random
import segmentation_models_pytorch as smp
import torch.optim as optim
import torch.nn as nn
from sklearn.metrics import jaccard_score


random.seed(42)
np.random.seed(42)
torch.manual_seed(42)

num_classes = 7
batch_size = 4
img_root = "/home/ealam/JHIR_Hip_Knee_Datasets/Hip/Images"
mask_root = "/home/ealam/JHIR_Hip_Knee_Datasets/Hip/Annotations"
metadata_path = "/home/ealam/JHIR_Hip_Knee_Datasets/Hip/segmentation_with_racegender.csv"


test_augmentations = A.Compose([
    A.Resize(height=512, width=512),
    A.Normalize(mean=(0.485, 0.485, 0.485), std=(0.229, 0.229, 0.229)),
    ToTensorV2(),
])

class MulticlassHipSegmentationDataset(Dataset):
    def __init__(self, img_root, mask_root, metadata_df, paired_files, num_classes, transforms=None, preprocessing=None):
        self.img_root = img_root
        self.mask_root = mask_root
        self.metadata_df = metadata_df
        self.paired_files = paired_files
        self.num_classes = num_classes
        self.transforms = transforms
        self.preprocessing = preprocessing

    def __len__(self):
        return len(self.paired_files)

    def __getitem__(self, idx):
        image_file, mask_file = self.paired_files[idx]
        if not os.path.exists(os.path.join(self.mask_root, mask_file)):
            return None

        dicom_image = pydicom.dcmread(os.path.join(self.img_root, image_file))
        image = dicom_image.pixel_array.astype(np.float32)
        image = cv2.resize(image, (512, 512))


        image = np.stack([image] * 3, axis=-1)

        annotation = nib.load(os.path.join(self.mask_root, mask_file))
        annotation_data = annotation.get_fdata()
        if len(annotation_data.shape) == 3:
            annotation_data = annotation_data[:, :, 0]

        annotation_data = self.calculate_flipped_rotated_mask(annotation_data)
        annotation_data = cv2.resize(annotation_data, (512, 512))

        if self.transforms is not None:
            transformed = self.transforms(image=image, mask=annotation_data)
            image = transformed["image"]
            annotation_data = transformed["mask"]

        annotation_data_onehot = self.one_hot_encode(annotation_data)

        if self.preprocessing is not None:
            transformed = self.preprocessing(image=image, mask=annotation_data_onehot)
            image = transformed["image"]
            annotation_data_onehot = transformed["mask"]

        patient_id = int(float(image_file.split(".")[0]))
        racegender_info = self.metadata_df.loc[self.metadata_df['id'] == patient_id]['racegender'].values
        racegender = racegender_info[0] if racegender_info.size > 0 else 'Unknown'

        return image, annotation_data_onehot, racegender

    def one_hot_encode(self, mask):
        one_hot_mask = np.zeros((self.num_classes, *mask.shape), dtype=np.float32)
        for class_idx in range(self.num_classes):
            one_hot_mask[class_idx][mask == class_idx] = 1.0
        return one_hot_mask

    def calculate_flipped_rotated_mask(self, mask):
        rotated_mask = cv2.rotate(mask, cv2.ROTATE_90_CLOCKWISE)
        flipped_rotated_mask = cv2.flip(rotated_mask, 1)
        return flipped_rotated_mask

metadata_df = pd.read_csv(metadata_path)

image_files = sorted(os.listdir(img_root))
mask_files = sorted(os.listdir(mask_root))

paired_files = []

for image_file in image_files:
    image_id = os.path.splitext(image_file)[0]
    mask_file = f"{image_id}.nii.gz"
    if mask_file in mask_files:
        paired_files.append((image_file, mask_file))

random.shuffle(paired_files)

train_size = int(0.7 * len(paired_files))
valid_size = int(0.1 * len(paired_files))
test_size = len(paired_files) - train_size - valid_size

train_pairs = paired_files[:train_size]
valid_pairs = paired_files[train_size:train_size + valid_size]
test_pairs = paired_files[train_size + valid_size:]

train_set = MulticlassHipSegmentationDataset(
    img_root, mask_root, metadata_df, train_pairs, num_classes,
    transforms=test_augmentations
)

def train_manet_model(num_epochs=100, encoder_name="resnet18"):

    model = smp.MAnet(
        encoder_name=encoder_name,
        encoder_weights="imagenet",
        in_channels=3,
        classes=num_classes,
    )

    device = torch.device("cpu")
    model.to(device)

    optimizer = optim.Adam(model.parameters(), lr=0.0001, weight_decay=1e-5)
    criterion = nn.CrossEntropyLoss()

    valid_set = MulticlassHipSegmentationDataset(
        img_root, mask_root, metadata_df, valid_pairs, num_classes,
        transforms=test_augmentations
    )

    valid_loader = DataLoader(valid_set, batch_size=batch_size, shuffle=False, num_workers=2)

    for epoch in range(num_epochs):
        train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=2)

        model.train()
        train_iou_list = []

        for batch_idx, (images, masks, racegender) in enumerate(train_loader):
            if images is None:
                continue

            images, masks = images.to(device), masks.to(device)
            optimizer.zero_grad()
            outputs = model(images)

            predicted_masks = torch.argmax(outputs, dim=1)
            loss = criterion(outputs, masks.argmax(dim=1))
            loss.backward()
            optimizer.step()

            train_iou = jaccard_score(
                masks.argmax(dim=1).cpu().numpy().flatten(),
                predicted_masks.cpu().numpy().flatten(),
                average='micro'
            )
            train_iou_list.append(train_iou)

        model.eval()
        valid_iou_list = []

        for batch_idx, (images, masks, racegender) in enumerate(valid_loader):
            images, masks = images.to(device), masks.to(device)
            with torch.no_grad():
                outputs = model(images)
            predicted_masks = torch.argmax(outputs, dim=1)

            valid_iou = jaccard_score(
                masks.argmax(dim=1).cpu().numpy().flatten(),
                predicted_masks.cpu().numpy().flatten(),
                average='micro'
            )

            valid_iou_list.append(valid_iou)

        train_iou_avg = np.mean(train_iou_list)
        valid_iou_avg = np.mean(valid_iou_list)

        print(f"Epoch [{epoch + 1}/{num_epochs}] - Train IoU: {train_iou_avg:.4f} - Validation IoU: {valid_iou_avg:.4f}")


    torch.save(model.state_dict(), 'manet_model_Hip_init.pth')


    test_set = MulticlassHipSegmentationDataset(
        img_root, mask_root, metadata_df, test_pairs, num_classes,
        transforms=test_augmentations
    )

    test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False, num_workers=2)

    model.eval()
    test_iou_list = []

    for batch_idx, (images, masks, racegender) in enumerate(test_loader):
        images, masks = images.to(device), masks.to(device)
        with torch.no_grad():
            outputs = model(images)
        predicted_masks = torch.argmax(outputs, dim=1)

        test_iou = jaccard_score(
            masks.argmax(dim=1).cpu().numpy().flatten(),
            predicted_masks.cpu().numpy().flatten(),
            average='micro'
        )

        test_iou_list.append(test_iou)

    test_iou_avg = np.mean(test_iou_list)


    print("Model saved successfully.")


print("Training MAnet ..")
train_manet_model(encoder_name="resnet18")


Training MAnet ..
Epoch [1/100] - Train IoU: 0.0318 - Validation IoU: 0.0223
Epoch [2/100] - Train IoU: 0.0363 - Validation IoU: 0.0171
Epoch [3/100] - Train IoU: 0.0449 - Validation IoU: 0.0254
Epoch [4/100] - Train IoU: 0.0596 - Validation IoU: 0.0483
Epoch [5/100] - Train IoU: 0.0673 - Validation IoU: 0.0764
Epoch [6/100] - Train IoU: 0.0868 - Validation IoU: 0.1112
Epoch [7/100] - Train IoU: 0.1122 - Validation IoU: 0.1391
Epoch [8/100] - Train IoU: 0.1275 - Validation IoU: 0.1486
Epoch [9/100] - Train IoU: 0.1666 - Validation IoU: 0.1591
Epoch [10/100] - Train IoU: 0.1745 - Validation IoU: 0.1778
Epoch [11/100] - Train IoU: 0.2042 - Validation IoU: 0.1796
Epoch [12/100] - Train IoU: 0.2199 - Validation IoU: 0.1935
Epoch [13/100] - Train IoU: 0.2437 - Validation IoU: 0.2069
Epoch [14/100] - Train IoU: 0.2677 - Validation IoU: 0.2177
Epoch [15/100] - Train IoU: 0.2939 - Validation IoU: 0.2314
Epoch [16/100] - Train IoU: 0.2975 - Validation IoU: 0.2468
Epoch [17/100] - Train IoU: 0.3